In [1]:
import datetime
import os
import shutil

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.metrics import MeanAbsolutePercentageError, MeanAbsoluteError, RootMeanSquaredError

import definitions
from training import train, data
from training.loguniform import LogUniform
from training.stepuniform import StepUniform
from training.steploguniform import StepLogUniform
from scipy.stats.distributions import randint
import numpy as np
import pandas as pd

import altair as alt

#alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

# Missing Mass Regression
## W -> l nu

In [2]:
dataset = 'Wlnu'
target = 'W'

In [3]:
jigsaw_train, jigsaw_val, jigsaw_test = data.get_jigsaw(dataset=dataset, target=target)
x_train, y_train, x_val, y_val, x_test, y_test = data.get_datasets(dataset=dataset, target=target, scale=True)
print("x_train:")
print(x_train)
print(f"num training samples: {x_train.shape[0]}")
print(f"num validation samples: {x_val.shape[0]}")
print(f"num testing samples: {x_test.shape[0]}")

x_train:
           METx      METy   Lx_reco   Ly_reco   Lz_reco   Lm_reco
0      0.441085 -0.123268 -0.738416 -0.248920  0.645141 -0.000997
1      1.032623 -0.340998 -1.236841 -0.302691  0.932249 -0.000997
2     -0.308838 -1.228226  0.567540  1.111681 -1.675584 -0.000997
3     -0.858471  0.119029  1.874423  1.071938 -1.007796 -0.000997
4      0.403788  0.631750 -1.258448 -1.783140 -1.340958 -0.000997
...         ...       ...       ...       ...       ...       ...
79995 -1.070399 -0.769946  1.094526  0.742549 -0.650372 -0.000997
79996  0.787721  0.803089 -1.202470  0.582324 -0.045666 -0.000997
79997 -0.591679 -0.670812 -0.965388  0.241642 -2.009183 -0.000997
79998 -0.823566  0.650868  1.238392 -0.328990 -1.548584 -0.000997
79999  0.553779  0.479564 -2.746308  0.795546  1.272725 -0.000997

[80000 rows x 6 columns]
num training samples: 80000
num validation samples: 10000
num testing samples: 10000


## Dataset
Simple W -> l nu data samples are used. Below, the generated W mass (Wm_gen), jigsaw reconstructed W mass (Wm_reco) and the difference of the two are shown.

In [4]:
alt.Chart(y_test).mark_bar().encode(alt.X(f"{definitions.TARGETS[dataset][target][0]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")

alt.Chart(...)

In [5]:
alt.Chart(jigsaw_test).mark_bar().encode(alt.X(f"{definitions.JIGSAW_TARGETS[dataset][target][0]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")

alt.Chart(...)

In [6]:
jigsaw_difference = pd.DataFrame({'Wm_gen - Wm_reco': y_test.values[:, 0] - jigsaw_test[definitions.JIGSAW_TARGETS[dataset][target][0]].values})
alt.Chart(jigsaw_difference).mark_bar().encode(alt.X("Wm_gen - Wm_reco:Q", bin=alt.Bin(extent=[0, 100], step=5)), y="count()")

alt.Chart(...)

## Training
A simple fully-connected neural network is trained and tuned using random search.

In [7]:
def build_v2_model(hparams, input_shape):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    for _ in range(hparams['num_layers']):
        model.add(layers.Dense(units=hparams['num_units'],
                               activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        model.add(layers.Dropout(rate=hparams['dropout']))
    model.add(layers.Dense(1, dtype='float32'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hparams['learning_rate']),
        loss='mean_squared_error',
        metrics=[MeanAbsolutePercentageError(), MeanAbsoluteError(), RootMeanSquaredError()])
    return model

In [8]:
def random_search(build_fn, x, y, x_val, y_val, n, hp_rv, log_dir):
    best_model = None
    best_loss = float('Inf')
    for i in range(n):
        run_name = f'run{i}'
        run_dir = log_dir / run_name
        with tf.summary.create_file_writer(str(run_dir)).as_default():
            hparams = {k: v.rvs() for k, v in hp_rv.items()}
            hparams['run_num'] = i
            hp.hparams(hparams)
            model, loss, mape, mae, rmse = train_test_model(
                build_fn, x, y, x_val, y_val, hparams, run_dir)
            tf.summary.scalar('mean absolute percentage error', mape, step=1)
            tf.summary.scalar('mean absolute error', mae, step=1)
            tf.summary.scalar('root mean squared error', rmse, step=1)
            if loss < best_loss:
                best_model = model
                best_loss = loss
    best_model.save(str(log_dir / 'best_model.h5'))

In [9]:
hp_rv = {'num_layers': randint(1, 3),
             'num_units': StepUniform(start=10, num=20, step=10),
             'learning_rate': LogUniform(loc=-5, scale=4, base=10, discrete=False),
             'batch_size': StepLogUniform(start=5, num=4, step=1, base=2),
             'epochs': randint(10, 101),
             'dropout': StepUniform(start=0.0, num=2, step=0.5)}

## Results

In [48]:
log_dir = definitions.LOG_DIR / 'Wlnu' / f'{target}-v2'
model = tf.keras.models.load_model(str(log_dir / 'best_model.h5'))

In [49]:
y_pred = model.predict(x_test)[:,0]

In [45]:

chart_data = pd.DataFrame({'Wm_gen - Wm_reco': np.concatenate((y_test - y_pred, y_test - jigsaw_test[definitions.JIGSAW_TARGETS[dataset][target][0]].values)), 'Method': ['NN']*y_test.shape[0] + ['Jigsaw']*y_test.shape[0]})

alt.Chart(chart_data).mark_bar(opacity=0.7).encode(alt.X("Wm_gen - Wm_reco:Q", bin=alt.Bin(extent=[-40, 30], step=2)), y=alt.Y("count()", stack=None), color="Method")

alt.Chart(...)

In [46]:
print('Jigsaw:')
print('\tmae = ' + str(float(tf.keras.losses.MAE(y_test, jigsaw_test.values[:, 0]))))
print(f'\tmape = ' + str(float(tf.keras.losses.MAPE(y_test, jigsaw_test.values[:, 0]))))
print('\trmse = ' + str(float(tf.keras.losses.MSE(y_test, jigsaw_test.values[:, 0])**0.5)))
print('NN:')
print('\tmae = ' + str(float(tf.keras.losses.MAE(y_test, y_pred))))
print('\tmape = ' + str(float(tf.keras.losses.MAPE(y_test, y_pred))))
print('\trmse = ' + str(float(tf.keras.losses.MSE(y_test, y_pred)**0.5)))

Jigsaw:
	mae = 16.617048470402004
	mape = 20.40721814904974
	rmse = 25.327719402850715
NN:
	mae = 2.9097039699554443
	mape = 3.584566354751587
	rmse = 5.938936710357666


In [47]:
chart_data = pd.DataFrame({'Generator': y_test, 'Jigsaw': jigsaw_test[definitions.JIGSAW_TARGETS[dataset][target][0]].values, 'NN': y_pred})

alt.Chart(chart_data).transform_fold(['Generator', 'Jigsaw', 'NN'], as_=['Method', 'Wm']).mark_area(interpolate='step-after', line=True, opacity=0.7).encode(alt.X("Wm:Q", bin=alt.Bin(extent=[38, 102], step=1)), y=alt.Y("count()", stack=None, scale=alt.Scale(type='log')), color='Method:N')

alt.Chart(...)